In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial
import os

todo_items = []

# 레이아웃
layout_title = widgets.Layout(width='100%')
layout_field = widgets.Layout(width='100%')
layout_button = widgets.Layout(width='90px', height='36px')
layout_row = widgets.Layout(margin='4px 0')

# 전체 진행률
progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=widgets.Layout(width='40%', height='24px'))
progress_label = widgets.Label(value="(0%)", layout=widgets.Layout(margin='0 10px'))

# 리스트
list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    progress_bar.value = avg
    progress_label.value = f"({int(avg)}%)"

def refresh():
    list_box.children = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_field)
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='100%', height='20px'))
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=widgets.Layout(width='100%'))
        date = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_field)
        priority = widgets.Label(value=item['priority'], layout=layout_field)

        done_btn = widgets.Button(description='완료', layout=layout_button)
        del_btn = widgets.Button(description='삭제', layout=layout_button)
        for btn in (done_btn, del_btn):
            btn.style.button_color = '#6B4C3B'  # 진한 회갈색

        def on_slider_change(change, i=item):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        def mark_done(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh()

        def delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh()

        slider.observe(on_slider_change, names='value')
        done_btn.on_click(partial(mark_done, i=item))
        del_btn.on_click(partial(delete, i=item))

        # 세로 배치
        row = widgets.VBox([
            widgets.HBox([widgets.Label("할 일", layout=layout_field), title], layout=layout_row),
            widgets.HBox([widgets.Label("진행률", layout=layout_field), widgets.VBox([bar, slider])], layout=layout_row),
            widgets.HBox([widgets.Label("입력 마감일", layout=layout_field), date], layout=layout_row),
            widgets.HBox([widgets.Label("중요도", layout=layout_field), priority], layout=layout_row),
            widgets.HBox([widgets.Label("작업", layout=layout_field), widgets.HBox([done_btn, del_btn])], layout=layout_row)
        ], layout=widgets.Layout(border='1px solid #eee', padding='10px'))
        list_box.children += (row,)

# 입력부
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_field)
progress_input = widgets.IntSlider(min=0, max=100, step=5, value=0, layout=layout_field)
due_input = widgets.DatePicker(layout=layout_field)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_field)
add_btn = widgets.Button(description='추가', button_style='success', layout=layout_button)

def add(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh()

add_btn.on_click(add)

# 저장/불러오기
def save(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                update_overall_progress()
                refresh()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_button)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_button)
save_btn.on_click(save)
load_btn.on_click(load)

# UI 출력
display(widgets.VBox([
    widgets.Label(value='DAVID mk13 - 세로정렬 & 회갈색 버튼'),
    title_input,
    progress_input,
    due_input,
    priority_input,
    add_btn,
    widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([progress_bar, progress_label], layout=widgets.Layout(justify_content='center')),
    list_box
]))

update_overall_progress()
refresh()
